# Sensitivity Analysis

In [15]:
import numpy as np
from scipy import signal
from scipy import linalg
import control
import matplotlib.pyplot as plt
import pickle

In [16]:
OSR = 256      # oversample ratio
fb = 22050     # nyquist
fs = OSR*2*fb  # sampling frequency
ts = 1/fs      # sampling period

In [21]:
# file = open('delta_filter_ss.pickle', 'rb')
# [A,B,C,D,T0,Ts,f,ts] = pickle.load(file)
# file.close()

In [18]:
# ----------------------------------------------------------
def delta_bode(A,B,C,D,f,ts):
  q     = C.shape[0]
  p     = B.shape[1]
  fs    = 1/ts
  mag   = np.zeros((q,p,f.shape[0]))
  phz   = np.zeros((q,p,f.shape[0]))
  delta = (np.exp(1j*2*np.pi*(f/fs))-1)/ts

  for i in range(f.shape[0]):
      A_d = delta[i] * np.eye(A.shape[0]) - A
      h   = C @ linalg.solve(A_d, B) + D
      mag[:,:,i] = np.abs(h)
      phz[:,:,i] = 180*np.arctan2(np.imag(h),np.real(h))/np.pi
  return mag, phz

In [19]:
# ----------------------------------------------------------
def ss_concat_outputs(sys_0, sys_1):
  A = linalg.block_diag(sys_0.A, sys_1.A)
  B = np.vstack((sys_0.B       , sys_1.B))
  C = linalg.block_diag(sys_0.C, sys_1.C)
  D = np.vstack((sys_0.D       , sys_1.D))
  return control.ss(A,B,C,D)

## SD dIIR Sensitivity
def SD_dIIR_sensitivity(A,B,C,D,T0,Ts,f,ts):

In [22]:
def SD_dIIR_sensitivity(A,B,C,D,T0,Ts,f,ts):
  # %Sensitivity due to feedthrough coefficient
  B_beta = np.zeros((1, A.shape[1])).T
  B_beta[0][0] = 1
  B_beta = A @ T0 @ B_beta
  S_beta0 = control.ss(A,B_beta,C,1)

  # %Sensitivity due to numerator coefficients
  S_beta = control.ss(A.T,C.T,Ts.T @ T0.T,0)

  # %Sensitivity due to denominator coefficients
  H = control.ss(A,B,C,D)
  S_alpha = control.series(H, S_beta)
  S_bsys = ss_concat_outputs(S_beta0, S_beta)
  null_sys = control.ss(0, 0, 0, 0)
  S_asys = ss_concat_outputs(null_sys, S_alpha)

  [S_bmag,S_bphz] = delta_bode(S_bsys.A,S_bsys.B,S_bsys.C,S_bsys.D,f,ts)
  [S_amag,S_aphz] = delta_bode(S_asys.A,S_asys.B,S_asys.C,S_asys.D,f,ts)
  S_mag = np.squeeze(S_bmag + S_amag)
  S_phz = np.squeeze(S_bphz + S_aphz)

  return S_mag, S_phz


In [23]:
SD_dIIR_sensitivity(Ad,Bd,Cd,Dd,T0,Ts,f,ts)

C:\Users\qaztr\AppData\Local\Temp\ipykernel_15436\1179078351.py:12: LinAlgWarning: Ill-conditioned matrix (rcond=5.41202e-25): result may not be accurate.
  h   = C @ linalg.solve(A_d, B) + D
C:\Users\qaztr\AppData\Local\Temp\ipykernel_15436\1179078351.py:12: LinAlgWarning: Ill-conditioned matrix (rcond=5.41201e-25): result may not be accurate.
  h   = C @ linalg.solve(A_d, B) + D
C:\Users\qaztr\AppData\Local\Temp\ipykernel_15436\1179078351.py:12: LinAlgWarning: Ill-conditioned matrix (rcond=5.412e-25): result may not be accurate.
  h   = C @ linalg.solve(A_d, B) + D
C:\Users\qaztr\AppData\Local\Temp\ipykernel_15436\1179078351.py:12: LinAlgWarning: Ill-conditioned matrix (rcond=5.41199e-25): result may not be accurate.
  h   = C @ linalg.solve(A_d, B) + D
C:\Users\qaztr\AppData\Local\Temp\ipykernel_15436\1179078351.py:12: LinAlgWarning: Ill-conditioned matrix (rcond=5.41198e-25): result may not be accurate.
  h   = C @ linalg.solve(A_d, B) + D
C:\Users\qaztr\AppData\Local\Temp\ipykerne

C:\Users\qaztr\AppData\Local\Temp\ipykernel_15436\1179078351.py:12: LinAlgWarning: Ill-conditioned matrix (rcond=1.00139e-25): result may not be accurate.
  h   = C @ linalg.solve(A_d, B) + D
C:\Users\qaztr\AppData\Local\Temp\ipykernel_15436\1179078351.py:12: LinAlgWarning: Ill-conditioned matrix (rcond=9.71976e-26): result may not be accurate.
  h   = C @ linalg.solve(A_d, B) + D
C:\Users\qaztr\AppData\Local\Temp\ipykernel_15436\1179078351.py:12: LinAlgWarning: Ill-conditioned matrix (rcond=9.42803e-26): result may not be accurate.
  h   = C @ linalg.solve(A_d, B) + D
C:\Users\qaztr\AppData\Local\Temp\ipykernel_15436\1179078351.py:12: LinAlgWarning: Ill-conditioned matrix (rcond=9.13887e-26): result may not be accurate.
  h   = C @ linalg.solve(A_d, B) + D
C:\Users\qaztr\AppData\Local\Temp\ipykernel_15436\1179078351.py:12: LinAlgWarning: Ill-conditioned matrix (rcond=8.85241e-26): result may not be accurate.
  h   = C @ linalg.solve(A_d, B) + D
C:\Users\qaztr\AppData\Local\Temp\ipyker

(array([[9.95512036e-01, 9.95512026e-01, 9.95512015e-01, ...,
         9.99221497e-01, 9.99238697e-01, 9.99255561e-01],
        [5.95877531e-02, 6.01722337e-02, 6.07624653e-02, ...,
         9.44341242e-02, 9.26553766e-02, 9.09113791e-02],
        [7.91453617e-02, 7.99218416e-02, 8.07059602e-02, ...,
         4.99393015e-02, 4.89891854e-02, 4.80578885e-02],
        [1.06899679e-01, 1.07948391e-01, 1.09007420e-01, ...,
         8.59130262e-03, 8.34237248e-03, 8.10066666e-03],
        [3.85148581e-05, 3.85148619e-05, 3.85148658e-05, ...,
         1.16478302e-06, 1.15320251e-06, 1.14174188e-06]]),
 array([[ 1.24674072e-06,  1.25964702e-06,  1.27269970e-06, ...,
          1.02157870e+00,  1.01155450e+00,  1.00163029e+00],
        [ 1.75304968e+02,  1.75347959e+02,  1.75390507e+02, ...,
         -2.27652548e+01, -2.31204284e+01, -2.34773303e+01],
        [ 1.75763377e+02,  1.75804675e+02,  1.75845573e+02, ...,
          3.17405867e+02,  3.17265170e+02,  3.17120880e+02],
        [ 1.75636689